# The farmer's problem

_This problem is taken from Section 1.1 of the book Birge, J. R., & Louveaux,
F. (2011). Introduction to Stochastic Programming. New York, NY: Springer New
York. Paragraphs in quotes are taken verbatim._

## Problem description

> Consider an European farmer who specializes in raising wheat, corn, and sugar
beets on his 500 acres of land. During the winter, [they want] to decide how
much land to devote to each crop.

> The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are
needed for cattle feed. These amounts can be raised on the farm or bought
from a wholesaler. Any production in excess of the feeding requirement would
be sold.

> Over the last decade, mean selling prices have been \\\$170 and \\\$150 per
ton of wheat and corn, respectively. The purchase prices are 40% more than
this due to the wholesaler’s margin and transportation costs.

> Another profitable crop is sugar beet, which [they expect] to sell at
\\\$36/T; however, the European Commission imposes a quota on sugar beet
production. Any amount in excess of the quota can be sold only at \\\$10/T.
The farmer’s quota for next year is 6000 T."

> Based on past experience, the farmer knows that the mean yield on [their]
land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar
beets, respectively.

> **To introduce uncertainty**, assume some correlation among the yields of the
different crops. A very simplified representation of this would be to assume
that years are good, fair, or bad for all crops, resulting in above average,
average, or below average yields for all crops. To fix these ideas, _above_
and _below_ average indicate a yield 20% above or below the mean yield.

## Problem data

The area of the farm.

In [1]:
max_area = 500.0;

There are three crops:

In [2]:
crops = [:wheat, :corn, :sugar_beet];

Each of the crops has a different planting cost (\\\$/acre).

In [3]:
plant_cost = Dict(
    :wheat      => 150.0,
    :corn       => 230.0,
    :sugar_beet => 260.0
);

The farmer requires a minimum quantity of wheat and corn, but not of sugar
beet (tonnes).

In [4]:
min_qt = Dict(
    :wheat      => 200.0,
    :corn       => 240.0,
    :sugar_beet =>   0.0
);

In Europe, there is a quota system for producing crops. The farmer owns the
following quota for each crop (tonnes):

In [5]:
quota_max = Dict(
    :wheat      => 20_0000,
    :corn       => 20_0000,
    :sugar_beet => 6_000.0
);

The farmer can sell crops produced under the quota for the following amounts
(\\\$/tonne):

In [6]:
sell_quota = Dict(
    :wheat      => 170.0,
    :corn       => 150.0,
    :sugar_beet =>  36.0
);

If they sell more than their alloted quota, the farmer earns the following on
each tonne of crop above the quota (\\\$/tonne):

In [7]:
sell_above = Dict(
    :wheat      => 0.0,
    :corn       => 0.0,
    :sugar_beet => 10.0
);

The purchase prices for wheat and corn are 40% more than their sales price.
However, the description does not address the purchase price of sugar beet.
Therefore, we use a large value of \\\$1,000/tonne.

In [8]:
buy_price = Dict(
    :wheat      =>   238.0,
    :corn       =>   210.0,
    :sugar_beet => 1_000.0
);

On average, each crop has the following yield in tonnes/acre:

In [9]:
mean_yield = Dict(
    :wheat      =>  2.5,
    :corn       =>  3.0,
    :sugar_beet => 20.0
);

However, the yield is random. In good years, the yield is +20% above average,
and in bad years, the yield is -20% below average.

The scenarios are then defined

In [10]:
scenarios = [:good, :fair, :bad];

And the yield is adjusted accordingly to the scenario

In [11]:
yield_sto = Dict(
    :good => 0.2,
    :fair => 0,
    :bad  => -0.2
);

Defining the scenarios' probability

In [12]:
# Test both probability distributions (prob1 for item (b) and prob2 for item (c))
prob1 = Dict(
    :good => 1/3,
    :fair => 1/3,
    :bad  => 1/3
);

prob2 = Dict(
    :good => 0.25,
    :fair => 0.25,
    :bad  => 0.5
);

## Mathematical formulation (Mean yield problem)

Load solver, we use ```Cbc```, and define the optimisation model.

In [13]:
using Cbc, JuMP

fm_mean = Model(Cbc.Optimizer);

### Variables

Variables independent of weather realisation:

In [14]:
@variable(fm_mean, x[c in crops] >= 0);                                      # Planted area per crop

Variables dependent on weather realisation:

In [15]:
@variable(fm_mean, y[c in crops] >= 0)                                       # Quantity sold below the quota
@variable(fm_mean, w[c in crops] >= 0)                                       # Quantity sold above the quota (restricted by the max quota)
@variable(fm_mean, z[c in crops] >= 0);                                      # Quantity bought

### Constraints

Constraints independent of weather realisation:

In [16]:
@constraint(fm_mean,sum(x[c] for c in crops) <= max_area);                   # Max farming area

Constraints dependent on weather realisation:

In [17]:
@constraint(fm_mean, [c in crops],                                           # Constraint defined in crops
    x[c]*mean_yield[c] + z[c] -                                              # Production + bought
    y[c] - w[c] >=                                                           # Amount of crops sold below minus above quota
    min_qt[c])                                                               # Minimum required for cattle feeding

@constraint(fm_mean, [c in crops],                                           # Constraint defined in crops
    y[c] <= quota_max[c]);                                                   # Quantity below quota follows the limit

Costs independent of the weather: the cost of the plantation.

In [18]:
@expression(fm_mean, cost_det, sum(plant_cost[c]*x[c] for c in crops));      # Deterministic part of the objective

Costs dependent on weather realisation (based on the mean of stochastic realisations).

In [19]:
@expression(fm_mean, cost_sto,                                               # Stochastic part of the objective
            sum((buy_price[c].*z[c] -                                        # Cost for buying crops
            sell_quota[c].*y[c] - sell_above[c].*w[c]) for c in crops));     # Revenue from selling crops

### Objective function

In [20]:
@objective(fm_mean, Min, cost_det + cost_sto);

# Checking the model
print(fm_mean)

Min 150 x[wheat] + 230 x[corn] + 260 x[sugar_beet] + 238 z[wheat] - 170 y[wheat] + 210 z[corn] - 150 y[corn] + 1000 z[sugar_beet] - 36 y[sugar_beet] - 10 w[sugar_beet]
Subject to
 2.5 x[wheat] - y[wheat] - w[wheat] + z[wheat] >= 200.0
 3 x[corn] - y[corn] - w[corn] + z[corn] >= 240.0
 20 x[sugar_beet] - y[sugar_beet] - w[sugar_beet] + z[sugar_beet] >= 0.0
 x[wheat] + x[corn] + x[sugar_beet] <= 500.0
 y[wheat] <= 200000.0
 y[corn] <= 200000.0
 y[sugar_beet] <= 6000.0
 x[wheat] >= 0.0
 x[corn] >= 0.0
 x[sugar_beet] >= 0.0
 y[wheat] >= 0.0
 y[corn] >= 0.0
 y[sugar_beet] >= 0.0
 w[wheat] >= 0.0
 w[corn] >= 0.0
 w[sugar_beet] >= 0.0
 z[wheat] >= 0.0
 z[corn] >= 0.0
 z[sugar_beet] >= 0.0


Solving it

In [21]:
optimize!(fm_mean);

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 4 (-3) rows, 10 (-2) columns and 13 (-5) elements
0  Obj 0 Primal inf 160 (2) Dual inf 1795 (4)
0  Obj 0 Primal inf 160 (2) Dual inf 3.9999999e+10 (6)
5  Obj -118600
Optimal - objective value -118600
After Postsolve, objective -118600, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -118600 - 5 iterations time 0.012, Presolve 0.00
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.02



Storing results (rounded in 2 decimal digits)

In [22]:
sol_plant_mean = round.(value.(x),digits=2)                                     # Areas planted
sol_bought_mean = round.(value.(z),digits=2)                                    # Quantities bought
sol_sold_bq_mean = round.(value.(y),digits=2)                                   # Sold below quota
sol_sold_aq_mean = round.(value.(w),digits=2);                                  # Sold above quota

Printing optimal solution

In [23]:
for c in crops
    if sol_plant_mean[c]>0
        println("\n The area for $c planted was: ",sol_plant_mean[c]," acres")
    end
    if sol_bought_mean[c]>0
        println("The quantity of $c bought is: ", sol_bought_mean[c])
    end
    if sol_sold_bq_mean[c]>0
        println("The quantity of $c sold below quota is: ", sol_sold_bq_mean[c])
    end
    if sol_sold_aq_mean[c]>0
        println("The quantity of $c sold above quota is: ", sol_sold_aq_mean[c])
    end
end


 The area for wheat planted was: 120.0 acres
The quantity of wheat sold below quota is: 100.0

 The area for corn planted was: 80.0 acres

 The area for sugar_beet planted was: 300.0 acres
The quantity of sugar_beet sold below quota is: 6000.0


## Mathematical formulation (2-stages stochastic optimisation)

In [40]:
fm = Model(Cbc.Optimizer);

### Define $1^{st}$-stage problem (not dependent on weather realisations)

### Variables

In [41]:
@variable(fm, x[c in crops] >= 0);                                               # Planted area per crop

### Constraints

In [42]:
@constraint(fm,sum(x[c] for c in crops) <= max_area);                            # Max farming area

First-stage costs: the cost of the plantation.

In [43]:
@expression(fm, first_cost, sum(plant_cost[c]*x[c] for c in crops));             # Deterministic objective

### Define $2^{nd}$-stage problem (dependent of weather realisations)

Variables

In [44]:
@variable(fm, y[c in crops, s in scenarios] >= 0)                                # Quantity sold below the quota
@variable(fm, w[c in crops, s in scenarios] >= 0)                                # Quantity sold above the quota (restricted by the max quota)
@variable(fm, z[c in crops, s in scenarios] >= 0);                               # Quantity bought

Constraints

In [45]:
@constraint(fm, [c in crops, s in scenarios],                                    # Constraint defined in crops and scenarios
    x[c]*(mean_yield[c]*(1+yield_sto[s])) + z[c,s] -                             # Production + bought
    y[c,s] - w[c,s] >=                                                           # Amount of crops sold below and above quota
    min_qt[c])                                                                   # Minimum required for cattle feeding

@constraint(fm, [c in crops,s in scenarios],                                     # Constraint defined in crops and scenarios
    y[c,s] <= quota_max[c]);                                                     # Quantity below quota follows the limit

Second-stage costs: the expected value of the profit coming from selling crops.

In [46]:
@expression(fm, second_cost,
            sum((buy_price[c]*z[c,s] -
            sell_quota[c]*y[c,s] - sell_above[c]*w[c,s])*prob1[s] for c in crops, s in scenarios));

In [47]:
second_cost

79.33333333333333 z[wheat,good] - 56.666666666666664 y[wheat,good] + 79.33333333333333 z[wheat,fair] - 56.666666666666664 y[wheat,fair] + 79.33333333333333 z[wheat,bad] - 56.666666666666664 y[wheat,bad] + 70 z[corn,good] - 50 y[corn,good] + 70 z[corn,fair] - 50 y[corn,fair] + 70 z[corn,bad] - 50 y[corn,bad] + 333.3333333333333 z[sugar_beet,good] - 12 y[sugar_beet,good] - 3.333333333333333 w[sugar_beet,good] + 333.3333333333333 z[sugar_beet,fair] - 12 y[sugar_beet,fair] - 3.333333333333333 w[sugar_beet,fair] + 333.3333333333333 z[sugar_beet,bad] - 12 y[sugar_beet,bad] - 3.333333333333333 w[sugar_beet,bad]

### Objective function

In [32]:
@objective(fm, Min, first_cost + second_cost);

# Checking the model
print(fm)

Min 150 x[wheat] + 230 x[corn] + 260 x[sugar_beet] + 59.5 z[wheat,good] - 42.5 y[wheat,good] + 59.5 z[wheat,fair] - 42.5 y[wheat,fair] + 119 z[wheat,bad] - 85 y[wheat,bad] + 52.5 z[corn,good] - 37.5 y[corn,good] + 52.5 z[corn,fair] - 37.5 y[corn,fair] + 105 z[corn,bad] - 75 y[corn,bad] + 250 z[sugar_beet,good] - 9 y[sugar_beet,good] - 2.5 w[sugar_beet,good] + 250 z[sugar_beet,fair] - 9 y[sugar_beet,fair] - 2.5 w[sugar_beet,fair] + 500 z[sugar_beet,bad] - 18 y[sugar_beet,bad] - 5 w[sugar_beet,bad]
Subject to
 3 x[wheat] - y[wheat,good] - w[wheat,good] + z[wheat,good] >= 200.0
 3.6 x[corn] - y[corn,good] - w[corn,good] + z[corn,good] >= 240.0
 24 x[sugar_beet] - y[sugar_beet,good] - w[sugar_beet,good] + z[sugar_beet,good] >= 0.0
 2.5 x[wheat] - y[wheat,fair] - w[wheat,fair] + z[wheat,fair] >= 200.0
 3 x[corn] - y[corn,fair] - w[corn,fair] + z[corn,fair] >= 240.0
 20 x[sugar_beet] - y[sugar_beet,fair] - w[sugar_beet,fair] + z[sugar_beet,fair] >= 0.0
 2 x[wheat] - y[wheat,bad] - w[wheat,ba

**Solving it**

In [33]:
optimize!(fm);

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 10 (-9) rows, 24 (-6) columns and 33 (-15) elements
0  Obj 0 Primal inf 493.33333 (6) Dual inf 1705.25 (12)
0  Obj 0 Primal inf 493.33333 (6) Dual inf 1.2e+11 (14)
12  Obj -94525
Optimal - objective value -94525
After Postsolve, objective -94525, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective -94525 - 12 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



Storing results

In [34]:
sol_plant = round.(value.(x),digits=2)          # Areas planted
sol_bought = round.(value.(z),digits=2)         # Quantities bought
sol_sold_bq = round.(value.(y),digits=2)        # Sold below quota
sol_sold_aq = round.(value.(w),digits=2);       # Sold above quota

Printing optimal solution

In [35]:
for c in crops
    if sol_plant[c]>0
        println("\n The area for $c planted was: ",sol_plant[c]," acres")
    end
    for s in scenarios
        if sol_bought[c,s]>0
            println("The quantity of $c bought in a $s yield would be: ", sol_bought[c,s])
        end
        if sol_sold_bq[c,s]>0
            println("The quantity of $c sold below quota in a $s yield would be: ", sol_sold_bq[c,s])
        end
        if sol_sold_aq[c,s]>0
            println("The quantity of $c sold above quota in a $s yield would be: ", sol_sold_aq[c,s])
        end
    end
end


 The area for wheat planted was: 100.0 acres
The quantity of wheat sold below quota in a good yield would be: 100.0
The quantity of wheat sold below quota in a fair yield would be: 50.0

 The area for corn planted was: 100.0 acres
The quantity of corn sold below quota in a good yield would be: 120.0
The quantity of corn sold below quota in a fair yield would be: 60.0

 The area for sugar_beet planted was: 300.0 acres
The quantity of sugar_beet sold below quota in a good yield would be: 6000.0
The quantity of sugar_beet sold above quota in a good yield would be: 1200.0
The quantity of sugar_beet sold below quota in a fair yield would be: 6000.0
The quantity of sugar_beet sold below quota in a bad yield would be: 4800.0


## Comparison between stochastic and deterministic formulations

Differences between solutions

In [155]:
for c in crops
    if sol_plant[c] - sol_plant_mean[c]!=0
        println("\n The area difference for $c planted would be: ",sol_plant[c] - sol_plant_mean[c]," acres")
    else
        println("\n")
    end
    for s in scenarios
        if sol_bought[c,s] - sol_bought_mean[c]!=0
            println("The difference of $c bought in a $s yield would be: ", sol_bought[c,s] - sol_bought_mean[c])
        end
        if sol_sold_bq[c,s] - sol_sold_bq_mean[c]!=0
            println("The difference of $c sold below quota in a $s yield would be: ", sol_sold_bq[c,s] - sol_sold_bq_mean[c])
        end
        if sol_sold_aq[c,s] - sol_sold_aq_mean[c]!=0
            println("The difference of $c sold above quota in a $s yield would be: ", sol_sold_aq[c,s] - sol_sold_aq_mean[c])
        end
    end
end


 The area difference for wheat planted would be: -20.0 acres
The difference of wheat sold below quota in a fair yield would be: -50.0
The difference of wheat sold below quota in a bad yield would be: -100.0

 The area difference for corn planted would be: 20.0 acres
The difference of corn sold below quota in a good yield would be: 120.0
The difference of corn sold below quota in a fair yield would be: 60.0


The difference of sugar_beet sold above quota in a good yield would be: 1200.0
The difference of sugar_beet sold below quota in a bad yield would be: -1200.0
